In [ ]:
%load_ext sparksql_magic

In [ ]:
import os
import pandas as pd
import json

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [ ]:
DATA_BUCKET = "/data_bucket/mimic-iv-clinical-database-demo-on-fhir-2.0/mimic-fhir/"
for item in os.listdir(DATA_BUCKET):
    print(item)

In [ ]:
spark = SparkSession.builder.appName("FhirDataApplication").getOrCreate()

In [ ]:
resources = ["Encounter", "Patient"]
for resource in resources:
    with open(f"./schema/{resource}.json") as f:
        schema_read = json.loads(f.read())
        schema = T.StructType.fromJson(schema_read)
    df = spark.read.schema(schema).json(f'{DATA_BUCKET}/{resource}.ndjson')
    df.createOrReplaceTempView(resource)

# Test Dataset

In [ ]:
%%sparksql
select maritalStatus.coding.code[0] maritialStatus, count(*) cnt from patient
group by 1

In [ ]:
%%sparksql
select
    get_json_object(subject, "$.reference") patient,
    count(*) cnt
from encounter
group by 1
order by 2 desc
limit 10

In [ ]:
spark.stop()